# WEEK 4 Exercises - Making Queries

![ERD-E-COMMERCE](https://github.com/it-web-pro/django-week4/blob/main/images/WEEK3-ERD(e-commerce).png?raw=true)

## Instruction

1. สร้าง `virtual environment`
2. ติดตั้ง `django` และ `psycopg2` libraries
3. สร้างโปรเจคใหม่ใหม่ชื่อ`myshop`
4. จากนั้นให้ทำการ startapp ใหม่ชื่อ `shop`
5. สร้าง database ชื่อ `shop` ใน Postgres DB
6. ทำการเพิ่ม code ด้านล่างนี้ในไฟล์ `shop/models.py`
7. เพิ่ม **'shop'** ใน `settings.py`
8. ทำการ `makemigrations` และ `migrate`

```python
from django.db import models

# Create your models here.


class Customer(models.Model):
    first_name = models.CharField(max_length=150)
    last_name = models.CharField(max_length=200)
    email = models.CharField(max_length=150)
    address = models.JSONField(null=True)

class ProductCategory(models.Model):
    name = models.CharField(max_length=150)

class Product(models.Model):
    name = models.CharField(max_length=150)
    description = models.TextField(null=True, blank=True)
    remaining_amount = models.PositiveIntegerField(default=0)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    categories = models.ManyToManyField(ProductCategory)

class Cart(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    create_date = models.DateTimeField()
    expired_in = models.PositiveIntegerField(default=60)
    
class CartItem(models.Model):
    cart = models.ForeignKey(Cart, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Order(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    order_date = models.DateField()
    remark = models.TextField(null=True, blank=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    payment_date = models.DateField()
    remark = models.TextField(null=True, blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)

class PaymentItem(models.Model):
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    order_item = models.OneToOneField(OrderItem, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    
class PaymentMethod(models.Model):
    class MethodChoices(models.Choices):
        QR = "QR"
        CREDIT = "CREDIT"
    
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    method = models.CharField(max_length=15, choices=MethodChoices.choices)
    price = models.DecimalField(max_digits=10, decimal_places=2)
```

**จากนั้นให้ทำการ migrate และ run คำสั่งในไฟล์ `shop.sql` ใน PgAdmin4**

In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

### 1. ให้นักศึกษา Query ค้นหาข้อมูลมาแสดงให้ถูกต้องตามโจทย์

1.1 query หาข้อมูล `Order` ทั้งหมดที่เกิดขึ้นในเดือน `พฤษภาคม` และ`ราคา`ของ Order มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
ORDER ID:22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID:23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID:24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID:25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID:26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID:27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID:28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID:29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID:30, DATE: 2024-05-03, PRICE: 1690.00
ORDER ID:31, DATE: 2024-05-04, PRICE: 3280.00
```

In [10]:
from shop.models import *

orders = Order.objects.filter(payment__price__gt=1000)[:10]

for order in orders:
    print(f"ORDER ID:{order.id}, DATE: {order.order_date}")

ORDER ID:2, DATE: 2024-04-24
ORDER ID:4, DATE: 2024-04-25
ORDER ID:5, DATE: 2024-04-25
ORDER ID:6, DATE: 2024-04-25
ORDER ID:8, DATE: 2024-04-26
ORDER ID:9, DATE: 2024-04-26
ORDER ID:10, DATE: 2024-04-27
ORDER ID:12, DATE: 2024-04-27
ORDER ID:13, DATE: 2024-04-28
ORDER ID:14, DATE: 2024-04-28


1.2 query หาข้อมูล `Product` ที่มีคำลงท้ายว่า `features.` ในรายละเอียดสินค้า และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.
```

In [22]:
question_2 = Product.objects.filter(description__endswith="features.")

for i in question_2:
    print(f"PRODUCT ID: {i.id}, DESCRIPTION: {i.description}")

PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.


1.3 query หาข้อมูล `Product` ที่มีราคาสินค้าตั้งแต่ `5000.00` ขึ้นไป และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID: 14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID: 31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID: 54, NAME: Automatic Pet Feeder, PRICE: 7900.00
PRODUCT ID: 61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID: 62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID: 63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID: 64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID: 65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00
```

In [26]:
question_3 = Product.objects.filter(price__gt="5000")

for i in question_3:
    print(f"PRODUCT ID: {i.id}, NAME: {i.name}, PRICE: {i.price}")

PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID: 14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID: 31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID: 54, NAME: Automatic Pet Feeder, PRICE: 7900.00
PRODUCT ID: 61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID: 62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID: 63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID: 64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID: 65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00


In [27]:
print(question_3.query)

SELECT "shop_product"."id", "shop_product"."name", "shop_product"."description", "shop_product"."remaining_amount", "shop_product"."price" FROM "shop_product" WHERE "shop_product"."price" > 5000


1.4 query หาข้อมูล `Product` ที่มีราคาสินค้าน้อยกว่า `200.00` และมากกว่า `100.00` และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
```

In [35]:
question_4 = Product.objects.filter(
    price__lt="200",
    price__gt="100"
)

for i in question_4:
    print(f"PRODUCT ID: {i.id}, NAME: {i.name}, PRICE: {i.price:.2f}")

PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 66, NAME: Salvage the Bones, PRICE: 129.00


### 2. เพิ่ม ลบ แก้ไข สินค้า

#### หมวดหมู่สินค้า
- Information technology
- Electronics
- Clothing and Apparel
- Home Appliances
- Furniture
- Toys and Games
- Books and Media
- Pet Supplies
- Jewelry

In [37]:
# มันมีอยู่แล้ว

2.1 ให้เพิ่มสินค้าใหม่จำนวน 3 รายการ (0.5 คะแนน)

```txt
สินค้าที่ 1
ชื่อ: Philosopher's Stone (1997)
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 20
รายละเอียดซ: By J. K. Rowling.
ราคา: 790

สินค้าที่ 2
ชื่อ: Me Before You
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 40
รายละเอียดซ: A romance novel written by Jojo
ราคา: 390

สินค้าที่ 3
ชื่อ: Notebook HP Pavilion Silver
หมวดหมู่สินค้า: Information Technology และ Electronics
จำนวนคงเหลือ: 10
รายละเอียดซ: Display Screen. 16.0
ราคา: 20000
```

In [52]:
# class Product(models.Model):
#     name = models.CharField(max_length=150)
#     description = models.TextField(null=True, blank=True)
#     remaining_amount = models.PositiveIntegerField(default=0)
#     price = models.DecimalField(max_digits=10, decimal_places=2)
#     categories = models.ManyToManyField(ProductCategory)

books = ProductCategory.objects.get(name="Books and Media")

product_1 = Product(
    name="Philosopher's Stone (1997)", 
    description="By J. K. Rowling.", 
    remaining_amount=20,
    price=790
)
product_1.save()
product_1.categories.add(books)

In [53]:
product_2 = Product(
    name="Me Before You", 
    description="A romance novel written by Jojo", 
    remaining_amount=40,
    price=390
)
product_2.save()
product_2.categories.add(books)

In [55]:
# สินค้าที่ 3
# ชื่อ: Notebook HP Pavilion Silver
# หมวดหมู่สินค้า: Information Technology และ Electronics
# จำนวนคงเหลือ: 10
# รายละเอียดซ: Display Screen. 16.0
# ราคา: 20000

category_it = ProductCategory.objects.get(name="Information Technology")
category_elec = ProductCategory.objects.get(name="Electronics")

product_3 = Product(
    name="Notebook HP Pavilion Silver", 
    description="Display Screen. 16.0", 
    remaining_amount=10,
    price=200
)
product_3.save()
product_3.categories.add(category_it)
product_3.categories.add(category_elec)

2.2 แก้ไขชื่อสินค้า จาก `Philosopher's Stone (1997)` เป็น `Half-Blood Prince (2005)` (0.5 คะแนน)

In [57]:
philosopher = Product.objects.get(name="Philosopher's Stone (1997)")
philosopher.name = "Half-Blood Prince (2005)"
philosopher.save()

2.3 แก้ไขชื่อหมวดหมู่สินค้า จาก `Books and Media` เป็น `Books` (0.5 คะแนน)

In [60]:
books_and_media = ProductCategory.objects.get(name="Books and Media")
books_and_media.name = "Books"
books_and_media.save()

2.4 ลบสินค้าทุกตัวที่อยู่ในหมวดหมู่ `Books` (0.5 คะแนน)

In [63]:
books = ProductCategory.objects.get(name="Books")

products_book = Product.objects.filter(categories=books)

for i in products_book:
    print(i.name)

Salvage the Bones
Me Before You
